In [20]:
import os
import glob
import json

def clean_key(k):
    return k.split("_", 1)[1]

base_path = "../../../evaluations/lince/pos/dev/"

evaluation_paths = glob.glob(f"{base_path}/*.json")

models = {}

for path in evaluation_paths:
    name = os.path.basename(path).split(".")[0]
    with open(path) as f:
        model_evaluation = json.load(f)
        clean_evaluations = []
        for task in model_evaluation["evaluations"].keys():
            task_evaluations = model_evaluation["evaluations"][task]
            clean_evaluations = [
                {clean_key(k): v for k, v in ev.items()} 
                for ev in task_evaluations
            ]

            model_evaluation["evaluations"][task] = clean_evaluations
        models[name] = model_evaluation
        


In [21]:

results = []

for model, model_evaluation in models.items():
    print("="*50)
    print(model)
    for task, task_evaluations in model_evaluation["evaluations"].items():
        print(task)
        print(len(task_evaluations))
        for evaluation in task_evaluations:

            results.append({
                **{
                    "model": model,
                    "task": task,
                },
                **evaluation,
            })

robertuito-base-uncased
pos
2
robertuito-base-deacc
pos
2
robertuito-base-cased
pos
2


In [22]:
import pandas as pd

pd.options.display.float_format = '{:.3f}'.format

mean_df = pd.DataFrame(results).groupby(["model"]).mean()
std_df = pd.DataFrame(results).groupby(["model"]).std()
# Magia negra

#mean_df = mean_df.unstack(1)
#std_df = std_df.unstack(1)

cols = ["micro_f1", "macro_f1", "accuracy", "precision", "recall"]
other_f1_cols = [c for c in mean_df.columns if "f1" in c and c not in cols]

idx = mean_df.index

mean_df.loc[idx, cols].round(4).astype(str) + " ± " + std_df.loc[idx, cols].round(3).astype(str)

#mean_df.loc[idx, cols].sort_values(by=["micro_f1"], ascending=False)

,micro_f1,macro_f1,accuracy,precision,recall
model,,,,,
robertuito-base-cased,0.9727 ± 0.001,0.9099 ± 0.001,0.9727 ± 0.001,0.9727 ± 0.001,0.9727 ± 0.001
robertuito-base-deacc,0.9681 ± 0.002,0.9024 ± 0.002,0.9681 ± 0.002,0.9681 ± 0.002,0.9681 ± 0.002
robertuito-base-uncased,0.9703 ± 0.001,0.9049 ± 0.001,0.9703 ± 0.001,0.9703 ± 0.001,0.9703 ± 0.001


In [23]:
pd.options.display.max_columns = None
mean_df.loc[idx, cols + other_f1_cols].sort_values(by=["accuracy"], ascending=False)

,micro_f1,macro_f1,accuracy,precision,recall,ADJ_f1,ADP_f1,ADV_f1,AUX_f1,CONJ_f1,DET_f1,INTJ_f1,NOUN_f1,NUM_f1,PART_f1,PRON_f1,PROPN_f1,PUNCT_f1,SCONJ_f1,UNK_f1,VERB_f1,X_f1
model,,,,,,,,,,,,,,,,,,,,,,
robertuito-base-cased,0.973,0.910,0.973,0.973,0.973,0.896,0.968,0.927,0.942,0.996,0.978,0.956,0.974,0.985,0.981,0.984,0.988,1.000,0.914,1.000,0.980,0.000
robertuito-base-uncased,0.970,0.905,0.970,0.970,0.970,0.893,0.966,0.927,0.941,0.997,0.979,0.956,0.964,0.986,0.982,0.985,0.916,1.000,0.913,0.998,0.980,0.000
robertuito-base-deacc,0.968,0.902,0.968,0.968,0.968,0.889,0.966,0.923,0.942,0.996,0.976,0.948,0.964,0.984,0.982,0.982,0.916,0.999,0.893,1.000,0.979,0.000
